# Computer Architecture for Data Scientists: from the transistor to The Cloud

### Kevin Moore
** Data Science BIG, April 11 2017 **

It's an amazing time to be a data scientist or a computer-savvy scientist. My laptop is like a supercomputer from 20 years ago. And, more computing power than most of us can imagine is available to all of us in the public cloud.

About Me:
* CEO Quilt Data
* PhD Computer Architecture
* MP memory systems & DBMS 
* Oracle and Sun Labs
* **not a biologist**

** Goal today: **
Explain a little about how computers work and what makes them go fast. Understanding that will (hopefully) help you:
* write better software
* make more informed software choices

### Example: intersect genomic regions

Let's say for example that we want to take a published CRISPR library from the well-known Lander-Sabatini Lab and intersect it with the set of common genetic variants known to be associated with a medial condition (published as part of the CLINVAR dataset).

In [113]:
# Why is this so much faster?
def intersect_fast(a, b):
    matches = None

    a_chrs = a.groupby('Chromosome')
    b_chrs = b.groupby('Chromosome')
    
    for a_chr, a_grp in a_chrs:
        b_grp = b_chrs.get_group(a_row['Chromosome'])
        for idx, a_row in a_grp.iterrows():
            a_match = b_grp.loc[(b_grp['start'] < a_row['end']) & ((b_grp['end'] > a_row['start']))]
            matches = pd.concat([matches, a_match]) if matches is not None else a_match
    return matches

In [114]:
# than this?
def intersect_naive(a, b):
    matches = []
    for a in rowmajor_a:
        for b in rowmajor_b:
            if a[0] == b[0] and int(a[1]) < int(b[2]) and int(a[2]) > int(b[1]):
                matches.append((a, b))
    return matches

The code in these two examples perform the same genomic intersection. Both are implemented in Python and neither uses explicit multiprocessing. But, the second example runs more than 100x faster. I'll use these loops and problem to illustrate some of the central concepts and techniques in computer architecture and explain some techniques for writing fast code.

## Key Concepts in Computer Architecture: Latency and Bandwidth
### Latency
* how long a single unit of a resource takes to arrive
* affected by size and distance of components
* changes slowly with technology generation (like clock rate)  

### Bandwidth
* how much of a resource can be delivered per unit time
* can often 'buy' more with money/cost, power, area, etc.
* gets cheaper with tech

The architect's goal is to be limited by bandwidth and "buy" only the bandwidth needed for the given system. To get there, we try to **hide latency** (by doing something else instead of waiting).

## Computer Architect’s secret weapons
### Locality
Architects take advantage of locality to **reduce latency** by:
* using smaller structures (e.g. caches instead of memory)
* making fewer, larger lookups (e.g. on disk and in memory)

### Speculation    
Speculation is making a prediction with the ability to undo any actions if the prediction turns out to be wrong. Speculation allows architects to **hide latency** by designing processors to do useful work while waiting for long-latency results.

## Von Neumann Model (1945)
In many ways computers haven’t changed much since the late 1940’s. Jon Von Neumann is credited with introducing the stored program computer in 1945. It had the following basic set of components:

* Memory
* Memory Address Register (MAR)
* Memory Data Register (MDR)
* ALU/TEMP(registers)
* Control
* I/O

That first computer and computers today execute instructions by following the series of basic steps:
![title](instruction.jpg)

Since then, switches are much smaller and faster (now transistors on ICs). Computers got so much smaller, faster and cheaper they became common in offices and homes in the 1980s. Even since then, computers have gotten much, much, faster. For example:

| Intel 386 (1989) | MacbookPro/Intel Skylake (2016)|
| ------------ | :-----------: |
| 33 MHz  | 2.9 GHz |
| 275K transistors | more than 1B Transistors |
| 4.3 MIPS  | 317 GIPS ||

**74000x faster**
~100x faster clock  
~740x improved architecture (**parallelism**)

## But my macbook only has 4 cores, where does that parallelism come from?
### Instruction-Level Parallelism

You might think that most of that speedup comes from the transistors getting faster, but the clock rate is only 100-times faster. Most of the speedup--the other 740x comes from **parallelism**. My laptop is a 4-core so we can chalk up 4x speedup to parallism. But, that still leaves 185x speedup from internal parallelism enabled by improved computer architecture and lots more transistors.

* **Pipelining**
    + ~5x parallelism
    + example below

* **Superscalar**
    + ~2x more parallelism
    + Complicated, very cool, next lecture

* **SIMD/Vector Instructions**
    + 16x+ parallelism
    + only for very regular execution patterns

## Pipelined Execution

Each instruction takes multiple (5 or more in the example below), but multiple instructions are executing at the same time. The example below shows a simple processor pipeline with 5 stages. Instructions move right to left as they execute various stages.

![title](pipeline_stages.jpg)

Modern Intel processors have more complicated pipelines that are both **super-scalar** (fetch and execute multiple instructions per cycle) and **out-of-order** (the processor executes instructions as soon as they are ready possibly bypassing earlier waiting instructions).

But, to make that work procs need:
* **Caches**  
* **Branch prediction** (guessing)

## Caches 

Computer memory system is a hierarchy:
* Registers (many accesses per cycle)
* L1 Cache - usually 1 cycle (10s of KBs)
* L2 Cache - usually 2-3 cycles (100s of KBs)
* L3 Cache - 10s of cycles (MBs)
* Main memory - 100s of cycles (GBs)
* Virtual memory (SSD) - 1000s of cycles (100s of GBs)

Caches store a small subset of memory that is loaded on demand. When a processor loads a memory location, it looks in its closest/smallest caches first. If the memory address isn't cached, it issues a request to higher levels of memory system. When the data are returned, they're loaded into the cache with the expectation that they're likely to be accessed again. Most software has a small working set in memory that is accessed more frequently than other locations. Caches (ideally) hold that working set for the duration of a program and speedup most memory accesses.

Caches are designed to take advantage of two forms of locality: **temporal locality** (the same memory location is accessed repeatedly in a short period of time) and **spatial locality** (adjacent memory locations are accessed together).

### Branch Prediction

Instructions that change the flow of instructions (e.g. to re-execute the body of a loop) are called branches. To avoid stalling the processor's pipeline, the instructions following a branch need to be fetched before the branch itself is executed.

![title](pipeline-branch-pred.jpg)

Processors speculate (guess) whether or not the branch will be taken, and flush the pipeline if they're wrong. Flushes are only a little more expensive than not speculating and can help bring instructions and data into processor caches faster. From a software developer's perspective, our goal is to write software with control flow that's easy to guess (e.g., **short tight loops with little internal branching**).

## Back to biology…

Why is this so slow?

In [127]:
def intersect_naive(rowmajor_a, rowmajor_b):
    matches = []
    for a in rowmajor_a:
        for b in rowmajor_b:
            if a[0] == b[0] and int(a[1]) < int(b[2]) and int(a[2]) > int(b[1]):
                matches.append((a, b))
    return matches

### First some code to setup our experiments

In [115]:
import csv
import signal
import time

# Datasets used in the examples
# see https://quiltdata.com for more info
# about Quilt.
from quilt.data.kmoore import CAforDS

def handler(signum, frame):
    print('Stopped')
    signal.alarm(0)
    raise TimerException("Alarm")
    
class TimerException(Exception):
    pass

# Cut the examples off if they haven't finished
# at the end of the timer so you don't have to
# wait all day.
signal.signal(signal.SIGALRM, handler)
timer = 20

In [116]:
# CRISPR Library from Lander-Sabatini Lab:
# - gRNA Name
# - Gene
# - Chromosome
# - Start
# - Stop
# - Strand
# - Sequence
# - Sequence revComp
# - oligo_F
# - oligo_R
# - sgA1BG_1

with open(CAforDS.raw.lander(), 'r') as a_bed:
    bedreader = csv.reader(a_bed, delimiter='\t')
    rowmajor_a = [row for row in bedreader]


In [118]:
# Clinvar Common-and-Known
# - Chromosome
# - Position
# - RSID
# - Ref
# - Alt
# - Info
with open(CAforDS.raw.clinvar(), 'r') as b_bed:
    bedreader = csv.reader(b_bed, delimiter='\t')
    rowmajor_b = [row for row in bedreader if not row[0].startswith('#')]

### Naive Implementation: Nested Loops over rows

In [128]:
count = 0
a_len = len(rowmajor_a)
b_len = len(rowmajor_b)
outrows = []
signal.alarm(timer)
try:
    for a in rowmajor_a[1:]:
        # Read a row from A
        a_chr = a[2]
        a_start = int(a[3])
        a_end = int(a[4])
        for b in rowmajor_b[1:]:
            # Read a row from B
            b_chr = "chr%s" % b[0]
            b_pos = int(b[1])
            if a_chr == b_chr and a_start <= b_pos and b_pos <= a_end:
                outrows.append(b)
            count += 1
    print("Finished")
except TimerException:
    print("Excecuted %ss" % timer)
finally:
    signal.alarm(0)
    miterations = count/1000000.0
    pct = count/(a_len*b_len)*100.0
    print("Completed %sM iterations (%0.2f%%)" % (miterations, pct))

Stopped
Excecuted 20s
Completed 24.811452M iterations (2.36%)


### Why is this so slow?

On my laptop, I get about 2.5% through the calculation in 20s so the whole thing would take around 14 minutes when the fast version will execute in about 5s.

Problem 1: **Algorithm**

An all-to-all comparison requires A * B operations.

In [120]:
acount = len(rowmajor_a)
bcount = len(rowmajor_b)
print("%d x %d = %d (%.2fB)" % (acount, bcount, acount*bcount, (float(acount)*bcount/(10**9))))

181131 x 5807 = 1051827717 (1.05B)


### Smarter approach: Partition by chromosome

Only genomic regions on the same Chromosome could possibly overlap, so we can dramatically reduce the number of comparisons by partitioning the inputs by chromosome and only comparing each region to the subset in the opposite input that matches its chromosome.

In [61]:
cc_a = {}
cc_b = {}
# Count B by chromosome
for bidx, b in enumerate(rowmajor_b[1:]):
    chrm = "chr%s" % b[0]
    if chrm not in cc_b:
        cc_b[chrm] = 0
    cc_b[chrm] += 1 
    
# Count A by chromosome
for aidx, a in enumerate(rowmajor_a[1:]):
    chrm = a[2]
    if chrm not in cc_a:
        cc_a[chrm] = 0
    cc_a[chrm] += 1 

#print(cc_a.keys())
#print(cc_b.keys())

total = 0
for chrm, count in cc_b.items():
    total += count*cc_a[chrm] if chrm in cc_a else 0
print("%d (%dM) operations" % (total, total/1000000.0))

53839387 (53M) operations


Reduces the work needed from 1B operations to 53M (**20x**). 

In [121]:
count = 0
outrows = []
signal.alarm(timer)
parts = {}
try:
    # Partition B
    starttime = time.time()
    for bidx, b in enumerate(rowmajor_b[1:]):
        chrm = "chr%s" % b[0]
        if chrm not in parts:
            parts[chrm] = []
        parts[chrm].append(bidx)
    
    # Iterate through A
    for a in rowmajor_a[1:]:
        a_chr = a[2]
        a_start = int(a[3])
        a_end = int(a[4])
        if a_chr in parts:
            for bidx in parts[a_chr]: 
                b = rowmajor_b[bidx+1]
                b_chr = "chr%s" % b[0]
                b_pos = int(b[1])
                if a_start <= b_pos and b_pos <= a_end:
                    outrows.append((a, b))
                count += 1            
    endtime = time.time()
    matches = len(outrows)
    print("Done, found %s matches in %ds" % (matches, endtime-starttime))

except TimerException:
    print("Excecuted %ss" % timer)
finally:
    signal.alarm(0)
    miterations = count/1000000.0
    pct = count/total*100.0
    print("Completed {it}M iterations ({pct:.2f}%)".format(it=miterations, pct=pct))

Stopped
Excecuted 20s
Completed 20.872505M iterations (38.77%)


That's a lot faster, but not quite 20x faster because the partitioning takes time.

But, it's still **slow**. What else is going wrong?

Problem 2: **Interpretation**
Python is an interpreted language so even basic operations (e.g. adding a pair of numbers) becomes a much longer set of steps. But, Python has hooks for developers to call out to functions written in C, which is compiled into machine-native binary code. That lets them publish Python libraries where the inner loops run at full speed for most of the calculation.

Problem 3: **Inefficent use of caches**

### What happens to caches?

If we read genomic interval data into memory in the way it's stored in a BED file, each genomic interval record (chromosome, start, end, strand, score, etc.) will be stored contiguously. Running the code above, the processor will load its caches 64-bytes at a line. As a result, the caches fill up with lots of unused data (score, name, block starts, etc.).

![title](cache_pollution.jpg)


### Branches are hard to predict

Inside the processor, comparing strings is actually more complicated than it looks. The processor has to test if each string is ending after each character as well as testing if the characters from the inputs match. And, mixing the chromosome comparison with a comparison of start and stop only adds to the number of branches the processor has to predict on each iteration of the loop. 

### Alternative: columnar processing

Instead, we can represent the set of genomic intervals as a collection of arrays:
* Chromosome []
* Start []
* End []
* ...

![title](cache_columns.jpg)

Accessing the data as columnar arrays leads to:
* Better cache locality (only bring in what we need)
* Less latency (fewer misses)
* More efficient use of BW (only loading what we need)
* Tighter loops (less branching)

Pandas
====

Pandas is a popular and very useful data science package for Python. It stores and operates on data in columnar form. It's built on top of numpy, a package of optimized array operations for fast numeric processing in Python. Using Pandas is an easy to way to avoid interpretation overhead in Python and use columnar processing.

In [122]:
import pandas as pd

lander = CAforDS.df.lander()
clinvar = pd.read_csv(CAforDS.raw.clinvar(),
                      sep='\t',
                      comment='#',
                      names=['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO'])

In [123]:
a = clinvar
b = lander

# match chromosome notation (chr<N>)
a['Chromosome'] = a['CHROM'].apply(lambda x: "chr%s" %x)

In [124]:
print(a.columns)
print(b.columns)
a_len = len(a.index)
b_len = len(b.index)

Index(['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO',
       'Chromosome'],
      dtype='object')
Index(['gRNAname', 'gene', 'chr', 'start', 'stop', 'strand', 'sequence',
       'sequence_revComp', 'oligo_F', 'oligo_R'],
      dtype='object')


In [125]:
matches = []
count = 0
signal.alarm(timer)
try:
    starttime = time.time()
    for idx, a_row in a.iterrows():
        a_match = b.loc[(b['chr'] == a_row['Chromosome']) & (b['start'] <= a_row['POS'])
                        & ((a_row['POS'] <= b['stop']))]
        matches.append(a_match)
        count += b_len
    allmatches = pd.concat(matches)
    endtime = time.time()
    print("Done, found %s matches in %ds" % (len(allmatches), endtime-starttime))
except TimerException:
    print("Excecuted %ss" % timer)
finally:
    signal.alarm(0)
    miterations = count/1000000.0
    pct = count/(a_len*b_len)*100.0
    print("Completed %.2fM iterations (%0.2f%%)" % (miterations, pct))

Stopped
Excecuted 20s
Completed 210.65M iterations (20.03%)


That's a lot faster, but not enough to make up for a slow algorithm.

### Partition Large Table

In [126]:
matches = []
count = 0
signal.alarm(timer)
try:
    starttime = time.time()
    b_chr = b.groupby('chr')
    for idx, a_row in a.iterrows():
        chr_grp = b_chr.get_group(a_row['Chromosome'])
        a_match = chr_grp.loc[(chr_grp['start'] <= a_row['POS'])
                              & ((a_row['POS'] <= chr_grp['stop']))]
        if len(a_match.index) > 0:
            matches.append(a_match)
        count += len(chr_grp)
    allmatches = pd.concat(matches)
    endtime = time.time()
    print("Done, found %s matches in %ds" % (len(allmatches), endtime-starttime))
except TimerException:
    print("Excecuted %d iterations" % count)
finally:
    signal.alarm(0)
    miterations = count/1000000.0
    pct = count/total*100.0
    print("Completed %.2fM iterations (%0.2f%%)" % (miterations, pct))

Done, found 462 matches in 17s
Completed 53.86M iterations (100.03%)


### Partition Both Tables

In [111]:
count = 0
signal.alarm(timer)
matches = []
try:
    starttime = time.time()
    a_chrs = a.groupby('Chromosome')
    b_chrs = b.groupby('chr')
    
    for a_chr, a_grp in a_chrs:
        b_grp = b_chrs.get_group(a_chr)
        for idx, a_pos in a_grp['POS'].iteritems():
            a_match = b_grp.loc[(b_grp['start'] <= a_pos)
                                & ((a_pos <= b_grp['stop']))]
            if a_match.index.any():
                matches.append(a_match)
            count += len(b_grp.index)
    allmatches = pd.concat(matches)
    endtime = time.time()
    print("Done, found %s matches in %ds" % (len(allmatches.index), endtime-starttime))
except TimerException:
    print("Excecuted %d iterations" % count)
finally:
    signal.alarm(0)
    miterations = count/1000000.0
    print("Completed {it}M iterations".format(it=miterations))

Done, found 462 matches in 5s
Completed 53.85803M iterations


### Review

**Start with your algorithm**
Even very efficient implementations of inefficient algorithms can be much, much slower than switching to a better, more scalable algorithm.

**Optimize before multithreading**
Modern CPUs can execute many instructions in parallel if the code and inputs let them. Use optimized libraries whenever possible (I highly recommend: **Python + Pandas + Pyspark**). Look for ways to organize your code into tight, regular loops.

**Pay attention to locality**
Caches are essential for good processor speed so try blocking your loops to cache-size chunks. **Partitioning** is one very effective technique.

### The Cloud

What about working in **The Cloud?** Computer architecture principles still apply: pay careful attention to **latency** and **bandwidth**.

**Avoid moving data**
Communication to and from cloud-hosted servers is slow. Each message and response to and from  a cloud datacenter suffers from long latency so avoid algorithms that require lots of request and response actions. Data transfer in and out of cloud datacenters is also limited in **bandwidth**. Moving GB let alone TB of data in or out of the cloud is very slow. It's usually cheaper to send computation to a computer near the data than to send data to a far-away computer. It can also be cheaper and faster to compress files or data before transferring them.

**Use Columnar Databases for *Analytics***
There are dozens of databases to choose from that are now easily deployable or available as a service from various cloud providers. They're widely different in the way they store and process data and some are much better suited than others for a given application. For analytics and other querying, choose a columnar or vector-based database.

Choose databases like:
* Druid
* Redshift (Amazon)
* BigQuery (Google)
* SQLServer (Microsoft)
* Snowflake (on AWS)

Avoid:
* MongoDB
* MySQL
* Postgres

There are also now several query-in-place SQL engines that have very good performance:
* PrestoDB
* Hive
* Athena (Amazon hosted Presto)
* Drill

**Avoid SELECT * **
Those columnar databases are amazingly efficient at querying, but most software that connects to them uses an interface that's row-based. That means that pulling a large result out of even a very fast database can be slow. If that database is hosted in a cloud datacenter, getting the result back can take even longer. Instead, move the computation into the database or at least to a computer in the same datacenter.

### Cloud Guidelines

**Choose software and cloud services optimized for your use**
Use columnar execution for computation intensive applications like data analysis. Use update-focused databases to store rapidly changing datasets.

**Reduce data movement to and from The Cloud**
Your laptop can billions of operations in the time it takes to get a response from a cloud service and your processor can pull GB/s from its local memory and SSD drive, but you'll be lucky to a few MB/s from even the best Internet services.

In [10]:
clinvar = pd.read_csv(CAforDS.raw.clinvar(),
                      sep='\t',
                      comment='#',
                      names=['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO'])

In [28]:
count = 0
signal.alarm(timer)
matches = []
try:
    starttime = time.time()
    a_chrs = a.groupby('Chromosome')
    b_chrs = b.groupby('chr')
    
    for a_chr, a_grp in a_chrs:
        b_grp = b_chrs.get_group(a_chr)
        for idx, a_pos in a_grp['POS'].iteritems():
            a_match = b_grp.loc[(b_grp['start'] <= a_pos) & ((a_pos <= b_grp['stop']))]
            if len(a_match.index) > 0:
                matches.append(a_match)
            count += len(b_grp.index)
    allmatches = pd.concat(matches)
    endtime = time.time()
    print("Done, found %s matches in %ds" % (len(allmatches.index), endtime-starttime))
except TimerException:
    print("Excecuted %d iterations" % count)
finally:
    signal.alarm(0)
    miterations = count/1000000.0
    print("Completed {it}M iterations".format(it=miterations))

Done, found 462 matches in 5s
Completed 53.85803M iterations


In [32]:

def profile_match(a, b):
    count = 0
    matches = []
    a_chrs = a.groupby('Chromosome')
    b_chrs = b.groupby('chr')
    
    for a_chr, a_grp in a_chrs:
        b_grp = b_chrs.get_group(a_chr)
        for idx, a_pos in a_grp['POS'].iteritems():
            a_match = b_grp.loc[(b_grp['start'] <= a_pos) & ((a_pos <= b_grp['stop']))]
            if len(a_match.index) > 0:
                matches.append(a_match)
            count += len(b_grp.index)
    allmatches = pd.concat(matches)
    return allmatches

%prun -s cumulative profile_match(a,b)

In [ ]:
dan = pd.read_csv('dan_genome/genome_Dan_Webster_v4_Full_20170504123348.txt',
                      sep='\t',
                      comment='#',
                      names=['rsid', 'chromosome', 'position', 'genotype'],
                      dtype={'chromosome' : 'category'})